In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.datasets import load_iris  # For classification example, you can change to a different dataset.

# Load dataset (this example uses the Iris dataset)
data = load_iris()
X = data.data
y = data.target

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Combine models into a voting ensemble
ensemble_model = VotingClassifier(estimators=[('rf', model1), ('gb', model2)], voting='hard')  # 'soft' for probabilistic voting

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
y_pred = ensemble_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of ensemble model: {accuracy * 100:.2f}%')

# If you're working with regression, use mean_squared_error for evaluation instead
# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse:.2f}')


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris, load_boston
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

# Define a function to check if the task is classification or regression
def get_task_type(y):
    if len(np.unique(y)) < 20:  # If fewer than 20 unique values, treat as classification
        return 'classification'
    else:
        return 'regression'

# Load dataset (change the dataset as needed)
def load_data(task='classification'):
    if task == 'classification':
        data = load_iris()
    elif task == 'regression':
        data = load_boston()  # Boston housing dataset for regression tasks
    else:
        raise ValueError("Unsupported task type!")
    return data

# Define the models you want to use in the ensemble
def get_base_models(task='classification'):
    if task == 'classification':
        # Define classifiers
        model1 = RandomForestClassifier(n_estimators=100, random_state=42)
        model2 = GradientBoostingClassifier(n_estimators=100, random_state=42)
        model3 = SVC(probability=True, random_state=42)
        model4 = KNeighborsClassifier(n_neighbors=5)
        return [model1, model2, model3, model4]
    elif task == 'regression':
        # Define regressors
        model1 = RandomForestClassifier(n_estimators=100, random_state=42)  # Use RandomForest for regression
        model2 = GradientBoostingClassifier(n_estimators=100, random_state=42)  # Gradient Boosting for regression
        return [model1, model2]
    else:
        raise ValueError("Unsupported task type!")

# Function to train the ensemble model
def train_ensemble(X_train, y_train, base_models, task_type='classification'):
    # Voting classifier for classification tasks
    if task_type == 'classification':
        ensemble_model = VotingClassifier(estimators=[(f'model{i+1}', model) for i, model in enumerate(base_models)], voting='soft')
    # Use Bagging for regression/classification tasks (alternative to Voting)
    elif task_type == 'regression':
        ensemble_model = BaggingClassifier(base_estimator=base_models[0], n_estimators=50, random_state=42)
    else:
        raise ValueError("Unsupported task type!")
    
    # Fit the ensemble model
    ensemble_model.fit(X_train, y_train)
    return ensemble_model

# Function to evaluate the ensemble model
def evaluate_model(ensemble_model, X_test, y_test, task_type='classification'):
    y_pred = ensemble_model.predict(X_test)
    
    if task_type == 'classification':
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f'Accuracy of the ensemble model: {accuracy * 100:.2f}%')
        print(f'F1 Score of the ensemble model: {f1:.2f}')
    elif task_type == 'regression':
        mse = mean_squared_error(y_test, y_pred)
        print(f'Mean Squared Error of the ensemble model: {mse:.2f}')
    else:
        raise ValueError("Unsupported task type!")

# Main function to run the ensemble learning template
def ensemble_learning(task_type='classification'):
    # Load data
    data = load_data(task=task_type)
    X = data.data
    y = data.target
    
    # Check task type
    task = get_task_type(y)
    if task != task_type:
        raise ValueError(f"The provided task type ('{task_type}') does not match the dataset type ('{task}')!")
    
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Get base models for the ensemble
    base_models = get_base_models(task_type=task_type)
    
    # Train the ensemble model
    ensemble_model = train_ensemble(X_train, y_train, base_models, task_type=task_type)
    
    # Evaluate the ensemble model
    evaluate_model(ensemble_model, X_test, y_test, task_type=task_type)

# Run the ensemble learning for classification task
ensemble_learning(task_type='classification')

# To run the ensemble learning for regression task, use:
# ensemble_learning(task_type='regression')
